In [3]:
# Código para Descompactar o arquivo que será usado para o Tech Challenge 3 Fine-Tuning
# Tratamento inicial desconsiderando dados nullos ou vazios para as colunas title e content
# Salvar os dados em formato parquet para grandes bases

In [4]:
import zipfile
import json
import gzip # Importar a biblioteca gzip

import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
file_path = '/content/drive/MyDrive/Fiap/LF-Amazon-1.3M.raw.zip'

In [6]:
#Lista os arquivos dentro do zip
try:
    with zipfile.ZipFile(file_path, 'r') as zf:
        print("Arquivos dentro do ZIP:")
        for file_info in zf.infolist():
            print(file_info.filename)
except FileNotFoundError:
    print(f"Erro: O arquivo ZIP '{file_path}' não foi encontrado.")
except zipfile.BadZipFile:
    print(f"Erro: O arquivo '{file_path}' não é um arquivo ZIP válido.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

Arquivos dentro do ZIP:
LF-Amazon-1.3M/
LF-Amazon-1.3M/lbl.json.gz
LF-Amazon-1.3M/trn.json.gz
LF-Amazon-1.3M/filter_labels_test.txt
LF-Amazon-1.3M/tst.json.gz
LF-Amazon-1.3M/filter_labels_train.txt


In [7]:
# Descompactar o arquivo zip
file_path = '/content/drive/MyDrive/Fiap/LF-Amazon-1.3M.raw.zip'
extracted_data_trn = [] # Nova lista para armazenar os dados extraídos de trn.json.gz

try:
    with zipfile.ZipFile(file_path, 'r') as zf:
        # Caminho para o arquivo .gz dentro do ZIP
        gz_file_path_in_zip = 'LF-Amazon-1.3M/trn.json.gz'
        with zf.open(gz_file_path_in_zip, 'r') as gz_file:
            # Descompactar o arquivo .gz e ler linha por linha
            # Usar gzip.open com 'rt' para ler como texto e iterar por linhas
            with gzip.open(gz_file, 'rt', encoding='utf-8') as f:
                for line in f:
                    # Remover espaços em branco no início/fim da linha antes de tentar decodificar
                    stripped_line = line.strip()
                    if not stripped_line: # Pular linhas vazias
                        continue
                    try:
                        # Tenta carregar cada linha como um objeto JSON individual
                        data_item = json.loads(stripped_line)
                        # Extrai as chaves desejadas, verificando se existem
                        uid = data_item.get('uid')
                        title = data_item.get('title')
                        content = data_item.get('content')

                        # Adiciona os dados extraídos a uma lista, se pelo menos uma chave existir
                        # ou se a chave existe e não é None/vazia (dependendo da sua regra)
                        # Aqui, adicionamos se pelo menos uma das chaves existe e não é None
                        if uid is not None or title is not None or content is not None:
                             # Adicionar os dados extraídos, garantindo que as chaves ausentes sejam None
                             extracted_data_trn.append({
                                 'uid': uid,
                                 'title': title,
                                 'content': content
                             })

                    except json.JSONDecodeError:
                        # Imprimir a linha que causou o erro para depuração
                        print(f"Aviso: Não foi possível decodificar a linha JSON do arquivo '{gz_file_path_in_zip}': {stripped_line[:200]}...") # Mostra apenas os primeiros 200 caracteres
                        continue # Pula para a próxima linha se a decodificação falhar


    print(f"Processamento de '{gz_file_path_in_zip}' concluído. Total de itens extraídos: {len(extracted_data_trn)}")
    # Opcional: Imprimir uma amostra dos dados extraídos
    # print(extracted_data_trn[:5])

except FileNotFoundError:
    print(f"Erro: O arquivo ZIP '{file_path}' não foi encontrado.")
except zipfile.BadZipFile:
    print(f"Erro: O arquivo '{file_path}' não é um arquivo ZIP válido.")
except KeyError:
    print(f"Erro: O arquivo '{gz_file_path_in_zip}' não foi encontrado dentro do arquivo ZIP.")
except gzip.BadGzipFile:
    print(f"Erro: O arquivo '{gz_file_path_in_zip}' dentro do ZIP não é um arquivo gzip válido.")
# O json.JSONDecodeError específico para json.load() não ocorrerá mais com a leitura linha a linha
# except json.JSONDecodeError:
#     print(f"Erro: Não foi possível decodificar o JSON do arquivo descompactado.")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")

Processamento de 'LF-Amazon-1.3M/trn.json.gz' concluído. Total de itens extraídos: 2248619


In [8]:
from pyspark.sql import SparkSession
import json
import os

# Inicializar a sessão Spark (se não estiver inicializada)
# É importante que o Spark tenha memória suficiente configurada, mas vamos tentar
# ler diretamente de um arquivo para evitar o problema de memória no driver.
spark = SparkSession.builder.appName("AmazonDataSpark").getOrCreate()

print("Sessão Spark inicializada.")

# Salvar a lista extracted_data_trn em um arquivo JSON Lines temporário
temp_json_file = "/tmp/extracted_data_trn.jsonl"
try:
    with open(temp_json_file, 'w', encoding='utf-8') as f:
        for item in extracted_data_trn:
            f.write(json.dumps(item) + '\n')
    print(f"Dados salvos temporariamente em '{temp_json_file}'.")

    # Ler o arquivo JSON Lines diretamente no Spark DataFrame
    # Spark pode ler arquivos JSON Lines usando read.json()
    df = spark.read.json(temp_json_file)

    print("DataFrame Spark criado com sucesso a partir do arquivo!")

    # Opcional: Exibir o esquema do DataFrame Spark
    df.printSchema()

    # Opcional: Exibir as primeiras linhas do DataFrame Spark
    df.show(5)

except Exception as e:
    print(f"Ocorreu um erro: {e}")
finally:
    # Limpar o arquivo temporário (opcional, mas boa prática)
    if os.path.exists(temp_json_file):
        os.remove(temp_json_file)
        pass

Sessão Spark inicializada.
Dados salvos temporariamente em '/tmp/extracted_data_trn.jsonl'.
DataFrame Spark criado com sucesso a partir do arquivo!
root
 |-- content: string (nullable = true)
 |-- title: string (nullable = true)
 |-- uid: string (nullable = true)

+--------------------+--------------------+----------+
|             content|               title|       uid|
+--------------------+--------------------+----------+
|High quality 3 la...|Girls Ballet Tutu...|0000031909|
|                    |Adult Ballet Tutu...|0000032034|
|                    |The Way Things Wo...|0000913154|
|Judith Kerr&#8217...|       Mog's Kittens|0001360000|
|                    |Misty of Chincote...|0001381245|
+--------------------+--------------------+----------+
only showing top 5 rows



In [9]:
df.createOrReplaceTempView("amazon_data")
# Executar uma consulta SQL para filtrar registros com 'content' não nulo e não vazio
df_filtered = spark.sql("""
        SELECT distinct title, content
        FROM amazon_data
        WHERE 1=1
        and content IS NOT NULL  AND trim(content) != ''
        and title IS NOT NULL AND trim(title) != ''
        and len(title) > 3
        and len(content) > 4
    """)


In [10]:
df_filtered.createOrReplaceTempView("df_filtered")
#Processamento de 'LF-Amazon-1.3M/trn.json.gz' concluído. Total de itens extraídos: 2248619
df_filtered.count()
#Total de registro após o distinct 1364345

1364345

In [11]:
# Salvar o DataFrame df_filtered em formato Parquet no Google Drive
output_path_parquet = '/content/drive/MyDrive/Fiap/amazon_data_parquet'

# Spark pode escrever DataFrames em vários formatos, incluindo Parquet.
# Usaremos o modo 'overwrite' para substituir o arquivo/diretório se já existir.
try:
    df_filtered.write.mode('overwrite').parquet(output_path_parquet)
    print(f"DataFrame df_filtered salvo com sucesso em '{output_path_parquet}' em formato Parquet.")

except Exception as e:
    print(f"Ocorreu um erro ao salvar o DataFrame em formato Parquet: {e}")

DataFrame df_filtered salvo com sucesso em '/content/drive/MyDrive/Fiap/amazon_data_parquet' em formato Parquet.
